## Indlejring

I vores tidligere eksempel arbejdede vi med høj-dimensionelle bag-of-words vektorer med længden `vocab_size`, og vi konverterede eksplicit fra lav-dimensionelle positionsrepræsentationsvektorer til sparsomme one-hot repræsentationer. Denne one-hot repræsentation er ikke hukommelseseffektiv, og derudover behandles hvert ord uafhængigt af hinanden, dvs. one-hot kodede vektorer udtrykker ikke nogen semantisk lighed mellem ord.

I denne enhed vil vi fortsætte med at udforske **News AG** datasættet. For at starte, lad os indlæse dataene og hente nogle definitioner fra den tidligere notebook.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Hvad er embedding?

Ideen med **embedding** er at repræsentere ord som lavdimensionelle tætte vektorer, der på en eller anden måde afspejler et ords semantiske betydning. Vi vil senere diskutere, hvordan man bygger meningsfulde word embeddings, men for nu kan vi bare tænke på embeddings som en måde at reducere dimensionaliteten af en ordvektor.

Så en embedding-lag vil tage et ord som input og producere en outputvektor med en specificeret `embedding_size`. På en måde minder det meget om et `Linear` lag, men i stedet for at tage en one-hot kodet vektor, vil det kunne tage et ordnummer som input.

Ved at bruge embedding-laget som det første lag i vores netværk kan vi skifte fra bag-of-words til en **embedding bag**-model, hvor vi først konverterer hvert ord i vores tekst til den tilsvarende embedding og derefter beregner en eller anden aggregeringsfunktion over alle disse embeddings, såsom `sum`, `average` eller `max`.

![Billede, der viser en embedding-klassifikator for fem sekvensord.](../../../../../translated_images/da/embedding-classifier-example.b77f021a7ee67eee.webp)

Vores klassifikator-neurale netværk vil starte med et embedding-lag, derefter et aggregeringslag og en lineær klassifikator ovenpå:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Håndtering af variabel sekvensstørrelse

Som et resultat af denne arkitektur skal minibatches til vores netværk oprettes på en bestemt måde. I den tidligere enhed, hvor vi brugte bag-of-words, havde alle BoW-tensore i en minibatch samme størrelse `vocab_size`, uanset den faktiske længde af vores tekstsekvens. Når vi skifter til ordindlejring, vil vi ende med et variabelt antal ord i hver tekstprøve, og når vi kombinerer disse prøver til minibatches, bliver vi nødt til at anvende noget padding.

Dette kan gøres ved hjælp af den samme teknik, hvor vi leverer `collate_fn`-funktionen til datakilden:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Træning af embedding-klassifikator

Nu hvor vi har defineret en passende dataloader, kan vi træne modellen ved hjælp af den træningsfunktion, vi har defineret i den forrige enhed:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Bemærk**: Vi træner kun på 25k poster her (mindre end en fuld epoke) for at spare tid, men du kan fortsætte træningen, skrive en funktion til at træne over flere epoker og eksperimentere med læringsrateparameteren for at opnå højere nøjagtighed. Du burde kunne nå en nøjagtighed på omkring 90%.


### EmbeddingBag-lag og repræsentation af sekvenser med variabel længde

I den tidligere arkitektur var vi nødt til at udfylde alle sekvenser til samme længde for at passe dem ind i en minibatch. Dette er ikke den mest effektive måde at repræsentere sekvenser med variabel længde på - en anden tilgang ville være at bruge en **offset**-vektor, som indeholder offsets for alle sekvenser, der er gemt i én stor vektor.

![Billede, der viser en offset-sekvensrepræsentation](../../../../../translated_images/da/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: På billedet ovenfor viser vi en sekvens af tegn, men i vores eksempel arbejder vi med sekvenser af ord. Dog forbliver det generelle princip om at repræsentere sekvenser med en offset-vektor det samme.

For at arbejde med offset-repræsentation bruger vi [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)-laget. Det ligner `Embedding`, men det tager indholdvektor og offset-vektor som input, og det inkluderer også et gennemsnitslag, som kan være `mean`, `sum` eller `max`.

Her er et modificeret netværk, der bruger `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

For at forberede datasættet til træning, skal vi levere en konverteringsfunktion, der vil forberede offset-vektoren:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Bemærk, at i modsætning til alle tidligere eksempler accepterer vores netværk nu to parametre: datavektor og offsetvektor, som har forskellige størrelser. Tilsvarende giver vores dataloader os også 3 værdier i stedet for 2: både tekst- og offsetvektorer leveres som funktioner. Derfor skal vi justere vores træningsfunktion en smule for at tage højde for dette:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantiske Embeddings: Word2Vec

I vores tidligere eksempel lærte modelens embedding-lag at kortlægge ord til vektorrepræsentation, men denne repræsentation havde ikke meget semantisk betydning. Det ville være rart at lære en sådan vektorrepræsentation, hvor lignende ord eller synonymer svarer til vektorer, der er tæt på hinanden i forhold til en eller anden vektordistance (f.eks. euklidisk distance).

For at opnå dette skal vi fortræne vores embedding-model på en stor samling af tekst på en specifik måde. En af de første metoder til at træne semantiske embeddings kaldes [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Det er baseret på to hovedarkitekturer, der bruges til at producere en distribueret repræsentation af ord:

 - **Continuous bag-of-words** (CBoW) — i denne arkitektur træner vi modellen til at forudsige et ord ud fra den omkringliggende kontekst. Givet ngrammet $(W_{-2},W_{-1},W_0,W_1,W_2)$ er målet for modellen at forudsige $W_0$ ud fra $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** er det modsatte af CBoW. Modellen bruger det omkringliggende vindue af kontekstord til at forudsige det aktuelle ord.

CBoW er hurtigere, mens skip-gram er langsommere, men gør et bedre stykke arbejde med at repræsentere sjældne ord.

![Billede, der viser både CBoW- og Skip-Gram-algoritmer til at konvertere ord til vektorer.](../../../../../translated_images/da/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

For at eksperimentere med word2vec embedding fortrænet på Google News-datasættet kan vi bruge **gensim**-biblioteket. Nedenfor finder vi de ord, der minder mest om 'neural'.

> **Note:** Når du først opretter ordvektorer, kan det tage noget tid at downloade dem!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Vi kan også beregne vektorindlejringer fra ordet, som skal bruges til at træne klassifikationsmodellen (vi viser kun de første 20 komponenter af vektoren for klarhed):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Den fantastiske ting ved semantiske indlejringer er, at du kan manipulere vektor-kodningen for at ændre semantikken. For eksempel kan vi bede om at finde et ord, hvis vektorrepræsentation ville være så tæt som muligt på ordene *konge* og *kvinde*, og så langt væk som muligt fra ordet *mand*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Både CBoW og Skip-Grams er "forudsigende" indlejringer, da de kun tager lokale kontekster i betragtning. Word2Vec udnytter ikke global kontekst.

**FastText** bygger videre på Word2Vec ved at lære vektorrepræsentationer for hvert ord og de karakter-n-grammer, der findes inden for hvert ord. Værdierne af repræsentationerne gennemsnitsberegnes derefter til én vektor ved hvert træningsskridt. Selvom dette tilføjer en del ekstra beregning til fortræningen, gør det det muligt for ordindlejringer at kode information på sub-ordniveau.

En anden metode, **GloVe**, udnytter ideen om en samforekomstmatrix og bruger neurale metoder til at dekomponere samforekomstmatricen til mere udtryksfulde og ikke-lineære ordvektorer.

Du kan eksperimentere med eksemplet ved at ændre indlejringer til FastText og GloVe, da gensim understøtter flere forskellige modeller for ordindlejring.


## Brug af Forudtrænede Embeddinger i PyTorch

Vi kan ændre eksemplet ovenfor for at forudfylde matricen i vores embedding-lag med semantiske embeddinger, såsom Word2Vec. Vi skal tage højde for, at ordforrådene fra de forudtrænede embeddinger og vores tekstkorpus sandsynligvis ikke vil matche, så vi vil initialisere vægtene for de manglende ord med tilfældige værdier:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Nu lad os træne vores model. Bemærk, at den tid, det tager at træne modellen, er betydeligt længere end i det tidligere eksempel, på grund af den større størrelse på indlejringslaget og dermed et meget højere antal parametre. Desuden kan vi på grund af dette være nødt til at træne vores model på flere eksempler, hvis vi vil undgå overtilpasning.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

I vores tilfælde ser vi ikke en stor stigning i nøjagtighed, hvilket sandsynligvis skyldes meget forskellige ordforråd.  
For at løse problemet med forskellige ordforråd kan vi bruge en af følgende løsninger:  
* Gen-træne word2vec-modellen på vores ordforråd  
* Indlæse vores datasæt med ordforrådet fra den forudtrænede word2vec-model. Det ordforråd, der bruges til at indlæse datasættet, kan specificeres under indlæsningen.  

Den sidstnævnte tilgang virker lettere, især fordi PyTorch `torchtext`-frameworket indeholder indbygget support til embeddings. Vi kan for eksempel oprette et GloVe-baseret ordforråd på følgende måde:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Indlæst ordforråd har følgende grundlæggende operationer:
* `vocab.stoi`-ordbogen giver os mulighed for at konvertere et ord til dets ordbogsindeks
* `vocab.itos` gør det modsatte - konverterer et tal til et ord
* `vocab.vectors` er arrayet af indlejringsvektorer, så for at få indlejringen af et ord `s` skal vi bruge `vocab.vectors[vocab.stoi[s]]`

Her er et eksempel på manipulation af indlejringer for at demonstrere ligningen **kind-man+woman = queen** (jeg var nødt til at justere koefficienten en smule for at få det til at fungere):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

For at træne klassifikatoren ved hjælp af disse indlejringer, skal vi først kode vores datasæt ved hjælp af GloVe-ordforrådet:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Som vi har set ovenfor, gemmes alle vektorembeddinger i `vocab.vectors` matrixen. Det gør det supernemt at indlæse disse vægte i vægtene for embeddingslaget ved hjælp af simpel kopiering:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Lad os nu træne vores model og se, om vi får bedre resultater:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

En af grundene til, at vi ikke ser en betydelig stigning i nøjagtighed, skyldes, at nogle ord fra vores datasæt mangler i den forudtrænede GloVe-ordforråd, og derfor bliver de i det væsentlige ignoreret. For at overvinde dette kan vi træne vores egne indlejringer på vores datasæt.


## Kontekstuelle Embeddinger

En af de væsentlige begrænsninger ved traditionelle forudtrænede embedding-repræsentationer som Word2Vec er problemet med ords betydningsafklaring. Selvom forudtrænede embeddinger kan fange noget af ordenes betydning i kontekst, bliver alle mulige betydninger af et ord kodet ind i den samme embedding. Dette kan skabe problemer i efterfølgende modeller, da mange ord, som for eksempel ordet 'play', har forskellige betydninger afhængigt af den kontekst, de bruges i.

For eksempel har ordet 'play' i disse to forskellige sætninger ret forskellige betydninger:
- Jeg gik til et **skuespil** i teatret.
- John vil gerne **lege** med sine venner.

De forudtrænede embeddinger ovenfor repræsenterer begge disse betydninger af ordet 'play' i den samme embedding. For at overvinde denne begrænsning skal vi bygge embeddinger baseret på **sproglige modeller**, som er trænet på et stort tekstkorpus og *forstår*, hvordan ord kan sættes sammen i forskellige kontekster. At diskutere kontekstuelle embeddinger er uden for rammerne af denne tutorial, men vi vender tilbage til dem, når vi taler om sproglige modeller i den næste enhed.



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
